In [1]:
%pip install xlrd openpyxl scikit-learn tqdm torchviz

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import tqdm as tqdm

np.version.full_version

'2.1.3'

In [2]:
hope_production_df = pd.read_excel('data.xlsx', sheet_name="HOPE PRODUCTION")
hope_storage_df = pd.read_excel('data.xlsx', sheet_name="HOPE STORAGE AFTER COOKING")

faith_production_df = pd.read_excel('data.xlsx', sheet_name="FAITH PRODUCTION")
faith_storage_df = pd.read_excel('data.xlsx', sheet_name="FAITH STORAGE AFTER COOKING")

hope_faith_df = pd.read_excel('data.xlsx', sheet_name="HOPE-FAITH PACKAGE WEIGHTS")
hope_faith_df.rename(columns={
    x: f"Sample {i + 1}" for i, x in 
    enumerate((x for x in hope_faith_df.columns if "Unnamed" in x))}, inplace=True)

hope_faith_df.columns

Index(['PRODUCT', 'DATE OF PACKING', 'EXPIRY DATE', 'AVERAGE WEIGHT (g)',
       'Sample 1', 'Sample 2', 'Sample 3', 'Sample 4', 'Sample 5', 'Sample 6',
       'Sample 7', 'Sample 8', 'Sample 9', 'Sample 10', 'Sample 11',
       'Sample 12', 'Sample 13', 'Sample 14', 'Sample 15', 'Sample 16',
       'Sample 17', 'Sample 18', 'Sample 19', 'Sample 20', 'Sample 21',
       'Sample 22', 'Sample 23', 'Sample 24', 'Sample 25', 'Sample 26',
       'Sample 27', 'Sample 28', 'Sample 29', 'Sample 30', 'Sample 31',
       'Sample 32'],
      dtype='object')

In [3]:
hope_df = hope_storage_df.merge(hope_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])
faith_df = faith_storage_df.merge(faith_production_df, right_on=["BATCH no.", "PRODUCTION DATE"], left_on=["BATCH no.", "BATCH INTO STORAGE"])
hope_df["PRODUCT"] = 5409
faith_df["PRODUCT"] = 5030

HOPE_SHELF_LIFE = 28
FAITH_SHELF_LIFE = 30

hope_df["ESTIMATED EXPIRY"] = hope_df["BATCH INTO STORAGE"] + pd.Timedelta(days=HOPE_SHELF_LIFE)
faith_df["ESTIMATED EXPIRY"] = faith_df["BATCH INTO STORAGE"] + pd.Timedelta(days=FAITH_SHELF_LIFE)

hope_pre_df = hope_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )
faith_pre_df = faith_df.merge(
        hope_faith_df, left_on=["ESTIMATED EXPIRY", "PRODUCT"], right_on=["EXPIRY DATE", "PRODUCT"]
    )


pre_df = pd.concat([hope_pre_df, faith_pre_df])
display(pre_df.columns)
# TODO: ONE PACKAGE HAS MORE THAN ONE PRODUCT
pre_df["PRODUCT AMOUNT EXPECTED"] = [1000 * row["BATCH WEIGHT (kg) AFTER COOKING"] / row["AVERAGE WEIGHT (g)"] for _, row in pre_df.iterrows()]

pre_df

Index(['BATCH no.', 'BATCH INTO STORAGE', 'BATCH OUT OF STORAGE',
       'BATCH WEIGHT LEAVING STORAGE (KG)', 'PRODUCTION DATE',
       'BATCH WEIGHT (kg) BEFORE COOKING', 'BATCH WEIGHT (kg) AFTER COOKING',
       'NOTES', 'PRODUCT', 'ESTIMATED EXPIRY', 'DATE OF PACKING',
       'EXPIRY DATE', 'AVERAGE WEIGHT (g)', 'Sample 1', 'Sample 2', 'Sample 3',
       'Sample 4', 'Sample 5', 'Sample 6', 'Sample 7', 'Sample 8', 'Sample 9',
       'Sample 10', 'Sample 11', 'Sample 12', 'Sample 13', 'Sample 14',
       'Sample 15', 'Sample 16', 'Sample 17', 'Sample 18', 'Sample 19',
       'Sample 20', 'Sample 21', 'Sample 22', 'Sample 23', 'Sample 24',
       'Sample 25', 'Sample 26', 'Sample 27', 'Sample 28', 'Sample 29',
       'Sample 30', 'Sample 31', 'Sample 32'],
      dtype='object')

,BATCH no.,BATCH INTO STORAGE,BATCH OUT OF STORAGE,BATCH WEIGHT LEAVING STORAGE (KG),PRODUCTION DATE,BATCH WEIGHT (kg) BEFORE COOKING,BATCH WEIGHT (kg) AFTER COOKING,NOTES,PRODUCT,ESTIMATED EXPIRY,...,Sample 24,Sample 25,Sample 26,Sample 27,Sample 28,Sample 29,Sample 30,Sample 31,Sample 32,PRODUCT AMOUNT EXPECTED
0,3752,2023-10-19,2023-10-19,194.4,2023-10-19,207.1,194.4,NaN,5409,2023-11-16,...,411,414,407,407,410,407,405,404.0,403.0,476.470588
1,3752,2023-10-19,2023-10-19,194.4,2023-10-19,207.1,194.4,NaN,5409,2023-11-16,...,400,401,403,403,400,402,405,403.0,400.0,480.704737
2,3753,2023-10-19,2023-10-19,194.1,2023-10-19,207.5,194.1,NaN,5409,2023-11-16,...,411,414,407,407,410,407,405,404.0,403.0,475.735294
3,3753,2023-10-19,2023-10-19,194.1,2023-10-19,207.5,194.1,NaN,5409,2023-11-16,...,400,401,403,403,400,402,405,403.0,400.0,479.962909
4,3754,2023-10-19,2023-10-19,193.4,2023-10-19,207.9,193.4,NaN,5409,2023-11-16,...,411,414,407,407,410,407,405,404.0,403.0,474.019608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30849,4181,2024-10-04,2024-10-07,258.6,2024-10-04,260.1,258.6,NaN,5030,2024-11-03,...,585,585,595,589,591,588,588,584.0,587.0,440.123391
30850,4182,2024-10-04,2024-10-07,257.0,2024-10-04,257.2,257.0,NaN,5030,2024-11-03,...,580,564,574,575,579,581,570,578.0,588.0,443.127324
30851,4182,2024-10-04,2024-10-07,257.0,2024-10-04,257.2,257.0,NaN,5030,2024-11-03,...,585,585,595,589,591,588,588,584.0,587.0,437.400277
30852,4183,2024-10-04,2024-10-06,168.2,2024-10-04,172.7,168.2,NaN,5030,2024-11-03,...,580,564,574,575,579,581,570,578.0,588.0,290.015626


In [4]:
INPUT_SIZE = 5
OUTPUT_SIZE = 1

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__(self)
        INTERNAL = 7
        self.l1 = nn.Linear(INPUT_SIZE, INTERNAL)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(INTERNAL, INTERNAL)
        self.lrelu = nn.LeakyReLU()
        self.l3 = nn.Linear(INTERNAL, OUTPUT_SIZE)
    
    def forward(self, x):
        return self.l3(self.lrelu(self.l2(self.relu(self.l1(x)))))

In [5]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df = pd.DataFrame()

df['product'] = le.fit_transform(pre_df['PRODUCT'])

df['time_in_storage'] = [x / np.timedelta64(1, 'D') for x in (pre_df["BATCH OUT OF STORAGE"] - pre_df["BATCH INTO STORAGE"]).values]
df['input_amount'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) BEFORE COOKING"].values.reshape(-1, 1))
df['cooking_out__storage_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT (kg) AFTER COOKING"].values.reshape(-1, 1))
df['storage_out__packaging_in'] = preprocessing.MinMaxScaler().fit_transform(pre_df["BATCH WEIGHT LEAVING STORAGE (KG)"].values.reshape(-1, 1))
df['product_amount_expected'] = preprocessing.MinMaxScaler().fit_transform(pre_df["PRODUCT AMOUNT EXPECTED"].values.reshape(-1, 1))

df

,product,time_in_storage,input_amount,cooking_out__storage_in,storage_out__packaging_in,product_amount_expected
0,1,0.0,0.653935,0.610111,0.610111,0.752764
1,1,0.0,0.653935,0.610111,0.610111,0.759485
2,1,0.0,0.655252,0.609163,0.609163,0.751597
3,1,0.0,0.655252,0.609163,0.609163,0.758308
4,1,0.0,0.656569,0.606951,0.606951,0.748874
...,...,...,...,...,...,...
34714,0,3.0,0.828449,0.812954,0.812954,0.695072
34715,0,3.0,0.818900,0.807899,0.807899,0.699840
34716,0,3.0,0.818900,0.807899,0.807899,0.690750
34717,0,2.0,0.540665,0.527330,0.527330,0.456813


In [6]:
class Data(torch.utils.data.Dataset):
    def __init__(self):
        self.data = df
        self.target = ["input_amount","cooking_out__storage_in", "storage_out__packaging_in"] 
    
    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index) -> tuple[torch.Tensor, torch.Tensor]:
        row = self.data.iloc[index]
        x = []
        y = []
        for k, v in row.items():
            if k in self.target:
                y.append(v)
            else:
                x.append(v)
        return tuple(map(torch.Tensor, [x, y]))

data = Data()
data[0]

(tensor([1.0000, 0.0000, 0.7528]), tensor([0.6539, 0.6101, 0.6101]))

In [7]:
INPUT_SIZE = len(data[0][0])
OUTPUT_SIZE = len(data[0][1])
INTERNAL = 7

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(INPUT_SIZE, INTERNAL)
        self.lrelu = nn.LeakyReLU()
        self.l2 = nn.Linear(INTERNAL, INTERNAL)
        self.relu = nn.ReLU()
        self.l3 = nn.Linear(INTERNAL, OUTPUT_SIZE)
    
    def forward(self, x):
        return self.l3(self.relu(self.l2((self.lrelu(self.l1(x))))))

INPUT_SIZE, INTERNAL, OUTPUT_SIZE

(3, 7, 3)

In [8]:
train_dataset, test_dataset = torch.utils.data.random_split(data, [.8, .2])

display(train_dataset)
display(test_dataset)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device.type
device = torch.device('cpu') 

In [10]:
model = Net().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scaler = torch.amp.GradScaler(device.type)
epochs = 100

In [11]:
for i in tqdm.tqdm(range(epochs)):
    for j, (x_train, y_train) in enumerate(train_dataset):
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        with torch.amp.autocast(device_type=device.type):
            y_pred = model(x_train)
            cost = criterion(y_pred, y_train)
        
        optimizer.zero_grad(set_to_none=True)
        scaler.scale(cost).backward()
        scaler.step(optimizer)
        scaler.update()
    if i % 1 == 0:
        torch.save(model, f"model-{i}.pt")

torch.save(model, "model.pt")

100%|██████████| 100/100 [38:49<00:00, 23.29s/it]


In [16]:
model_test = torch.load('model.pt')
model_test.eval()

losses = []
with torch.no_grad():
    for i, (x, y) in enumerate(test_dataset):
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x).to(device)
        loss = criterion(y_pred, y)
        losses.append(loss)
        print(loss)
max(losses)


C:\Users\pix\AppData\Local\Temp\ipykernel_21392\348806393.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_test = torch.load('model.pt')


[tensor(3.6893e-05), tensor(0.0001), tensor(4.8867e-05), tensor(8.9665e-05), tensor(9.4445e-05), tensor(1.1210e-06), tensor(9.1163e-05), tensor(8.4304e-05), tensor(8.6058e-06), tensor(0.0001), tensor(2.5868e-05), tensor(1.5684e-05), tensor(0.0002), tensor(6.0221e-06), tensor(0.0001), tensor(6.9130e-05), tensor(5.4539e-05), tensor(3.2491e-05), tensor(2.5521e-05), tensor(1.6386e-06), tensor(6.9325e-05), tensor(2.9139e-06), tensor(4.3666e-06), tensor(0.0002), tensor(0.0003), tensor(1.2241e-05), tensor(3.2201e-05), tensor(2.4537e-06), tensor(0.0002), tensor(1.4898e-05), tensor(0.0002), tensor(5.2685e-05), tensor(0.0001), tensor(0.0001), tensor(1.3547e-05), tensor(9.7870e-07), tensor(0.0006), tensor(2.5222e-05), tensor(9.9775e-05), tensor(0.0008), tensor(2.2858e-05), tensor(2.0268e-06), tensor(0.0001), tensor(0.0002), tensor(8.1112e-06), tensor(1.7693e-05), tensor(0.0001), tensor(9.2154e-05), tensor(2.7240e-05), tensor(1.2528e-05), tensor(7.5150e-07), tensor(1.0326e-05), tensor(0.0002), ten